In [4]:
import pandas as pd
import numpy as np

Will have to make some code here to handle the original file format if use an updated version  
Think I manually dropped the first week 1 - 11 games in excel  
Also deleted some columns 
Manually gave the home/away column name - OG data has no column name for it

### Create home and away team columns from data
Data comes in with winner and loser teams vs home and away  
Have to use the home/away column

In [3]:

def get_home_and_away_columns(sched_df):
    sched_df['home'] = np.where(
        sched_df['home/away'] == '@', 
        sched_df['Loser/tie'], 
        sched_df['Winner/tie']
    )
    sched_df['away'] = np.where(
        sched_df['home/away'] == '@', 
        sched_df['Winner/tie'], 
        sched_df['Loser/tie']
    )

    return sched_df

### Check if any home and away teams are the same value
If ```True in Same``` returns True than have an issue 

### Remove City from Team Name
Also convert to all lowercase

In [2]:

def get_short_team_name(full_team_name):
    short_name = full_team_name.rsplit(None, 1)[-1]
    lc_short_name = short_name.lower()
    return lc_short_name


In [6]:
sched_df['home'] = sched_df['home'].apply(lambda x: get_short_team_name(x))
sched_df['away'] = sched_df['away'].apply(lambda x: get_short_team_name(x))

### Get Home and Away Score 
Stored in original file as Points Winner (PtsW) and Points Loser (PtsL)

In [7]:
sched_df['home_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsL'], sched_df['PtsW'])
sched_df['away_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsW'], sched_df['PtsL'])


#### Drop unused columns

In [8]:
sched_df = sched_df.drop(columns=['Winner/tie','home/away','Loser/tie','PtsW','PtsL'])


### [datetime docs]('https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html')

## Create Game Time Column based on date and time columns

In [9]:
sched_df['GameTime'] = pd.to_datetime(sched_df['Date'] + 'T' + sched_df['Time'] + '-0500', utc=True )
print(sched_df.dtypes)
sched_df.head()

Week                        int64
Day                        object
Date                       object
Time                       object
home                       object
away                       object
home_score                  int64
away_score                  int64
GameTime      datetime64[ns, UTC]
dtype: object


,Week,Day,Date,Time,home,away,home_score,away_score,GameTime
0,13,Thu,2022-12-01,8:15 PM,patriots,bills,10,24,2022-12-02 01:15:00+00:00
1,13,Sun,2022-12-04,1:00 PM,texans,browns,14,27,2022-12-04 18:00:00+00:00
2,13,Sun,2022-12-04,1:00 PM,ravens,broncos,10,9,2022-12-04 18:00:00+00:00
3,13,Sun,2022-12-04,1:00 PM,lions,jaguars,40,14,2022-12-04 18:00:00+00:00
4,13,Sun,2022-12-04,1:00 PM,vikings,jets,27,22,2022-12-04 18:00:00+00:00


### Create Start and End Times based off Game Time
Start time is 4 days 1 hour before start of game  
End time is 1 hour before start of game

In [10]:
sched_df['SearchStartTime'] = sched_df['GameTime'] - pd.to_timedelta('4 days 01:00:00')
sched_df['SearchEndTime'] = sched_df['GameTime'] - pd.to_timedelta('01:00:00')
sched_df.head()

,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,13,Thu,2022-12-01,8:15 PM,patriots,bills,10,24,2022-12-02 01:15:00+00:00,2022-11-28 00:15:00+00:00,2022-12-02 00:15:00+00:00
1,13,Sun,2022-12-04,1:00 PM,texans,browns,14,27,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,2022-12-04 17:00:00+00:00
2,13,Sun,2022-12-04,1:00 PM,ravens,broncos,10,9,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,2022-12-04 17:00:00+00:00
3,13,Sun,2022-12-04,1:00 PM,lions,jaguars,40,14,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,2022-12-04 17:00:00+00:00
4,13,Sun,2022-12-04,1:00 PM,vikings,jets,27,22,2022-12-04 18:00:00+00:00,2022-11-30 17:00:00+00:00,2022-12-04 17:00:00+00:00


### Save to CSV

In [11]:

sched_df.to_csv('./2022_schedule_cleaned.csv', index=False)

In [6]:


def get_weekly_df(filepath, week):
    # import schedule
    sched_df = pd.read_csv(filepath)

    # drop columns
    sched_df = sched_df.drop(['YdsW','TOW','YdsL','TOL', 'Unnamed: 7'], axis=1)

    #rename home/away column    
    sched_df = sched_df.rename(columns={"Unnamed: 5": "home/away"})

    sched_df = get_home_and_away_columns(sched_df)

    # verify no home and away teams the same
    same = np.where(sched_df['home'] == sched_df['away'], True, False)
    if (True in same):
        print('error, same team name in home and away')
        return

    # remove city from team names
    sched_df['home'] = sched_df['home'].apply(lambda x: get_short_team_name(x))
    sched_df['away'] = sched_df['away'].apply(lambda x: get_short_team_name(x))
    
    # create home and away score columns frm ptsl& ptsw
    sched_df['home_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsL'], sched_df['PtsW'])
    sched_df['away_score'] = np.where(sched_df['home/away'] == '@', sched_df['PtsW'], sched_df['PtsL'])

    # drop remaining unused columns
    sched_df = sched_df.drop(columns=['Winner/tie','home/away','Loser/tie','PtsW','PtsL'])
    
    # create game time column from date and time col in UTC format
    sched_df['GameTime'] = pd.to_datetime(sched_df['Date'] + 'T' + sched_df['Time'] + '-0500', utc=True )

    # create search start and end time columns
    sched_df['SearchStartTime'] = sched_df['GameTime'] - pd.to_timedelta('4 days 01:00:00')
    sched_df['SearchEndTime'] = sched_df['GameTime'] - pd.to_timedelta('01:00:00')

    # save as csv
    save_filepath = f'./schedules/schedule_wk_{week}_cleaned.csv'

    sched_df.to_csv(save_filepath, index=False)

    return sched_df
    

In [7]:
pathname = './schedules/schedule_wk_14.csv'
week = '14'

wk14 = get_weekly_df(pathname, '14')
wk14



     


,Week,Day,Date,Time,home,away,home_score,away_score,GameTime,SearchStartTime,SearchEndTime
0,14,Thu,2022-12-08,8:15 PM,rams,raiders,17,16,2022-12-09 01:15:00+00:00,2022-12-05 00:15:00+00:00,2022-12-09 00:15:00+00:00
1,14,Sun,2022-12-11,1:00 PM,bills,jets,20,12,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
2,14,Sun,2022-12-11,1:00 PM,bengals,browns,23,10,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
3,14,Sun,2022-12-11,1:00 PM,cowboys,texans,27,23,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
4,14,Sun,2022-12-11,1:00 PM,lions,vikings,34,23,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
5,14,Sun,2022-12-11,1:00 PM,titans,jaguars,22,36,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
6,14,Sun,2022-12-11,1:00 PM,giants,eagles,22,48,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
7,14,Sun,2022-12-11,1:00 PM,steelers,ravens,14,16,2022-12-11 18:00:00+00:00,2022-12-07 17:00:00+00:00,2022-12-11 17:00:00+00:00
8,14,Sun,2022-12-11,4:20 PM,broncos,chiefs,28,34,2022-12-11 21:20:00+00:00,2022-12-07 20:20:00+00:00,2022-12-11 20:20:00+00:00
9,14,Sun,2022-12-11,4:25 PM,seahawks,panthers,24,30,2022-12-11 21:25:00+00:00,2022-12-07 20:25:00+00:00,2022-12-11 20:25:00+00:00
